In [107]:
from collections import Counter
import nltk
nltk.download('averaged_perceptron_tagger')
f = open('C:/Users/aanch/Desktop/Courses/NLP/Project/SemEval2018-Task3-master/SemEval2018-Task3-master/datasets/train/SemEval2018-T3-train-taskA.txt',encoding="utf8")
text = f.readlines()

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\aanch\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [117]:
tweets = []
for line in text:
    if not line.lower().startswith("tweet index"): # discard first line if it contains metadata
        #print("line: ",line)
        tweet = line.rstrip()
        tweets.append(tweet.split('\t')[2])

In [231]:
char_count =[]
word_count = []
wordmean = []

for tweet in tweets:
    char_count.append(len(tweet)) #get length of tweet text
    words = tweet.split() ##split tweet text into words
    word_count.append(len(words))
    word_length = []
    for i in words:
        word_length.append(len(i))
    wordmean.append(sum(word_length)/len(word_length))

In [155]:
from nltk.tokenize import TweetTokenizer
# reduce_len -- Replace repeated character sequences of length 3 or greater with sequences of length 3
# strip_handles -- Remove Twitter username handles from text (eg: @xyz)
tokenizer = TweetTokenizer(preserve_case=False, reduce_len=True, strip_handles=True).tokenize

In [123]:
for tweet in tweets:
    token = tokenizer(tweet)
    tags = nltk.pos_tag(token)
    counts = Counter(tag for word,tag in tags) #Counts only the tags

In [124]:
noun =[]
verb = []
adverb = []
adjective = []
for line in tweets:
    token= tokenizer(line)
    a = nltk.Text(token)
    tags=nltk.pos_tag(a)
    counts = Counter(tag for word,tag in tags)
    count_noun = counts['NN']+counts['NNS']+counts['NNP']+counts['NNPS']
    noun.append(count_noun)
    count_verb = counts['VB']+counts['VBD']+counts['VBG']+counts['VBN']+counts['VBP']+counts['VBZ']
    verb.append(count_verb)
    count_adverb = counts['RB']+counts['RBR']+counts['RBS']
    adverb.append(count_adverb)
    count_adjective = counts['JJ']+counts['JJS']+counts['JJR']
    adjective.append(count_adjective)

In [ ]:
print("count_noun: ", count_noun)
print("count_verb: ", count_verb)
print("count_adverb: ", count_adverb)
print("count_adjective: ", count_adjective)

In [200]:
nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import wordnet
from nltk.corpus import sentiwordnet as swn
nltk.download('sentiwordnet')
import re
from nltk.corpus import stopwords

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\aanch\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package sentiwordnet to
[nltk_data]     C:\Users\aanch\AppData\Roaming\nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!


In [201]:
lmtzr = WordNetLemmatizer()
tokenizer_regex = RegexpTokenizer(r'\w+')
pattern = '(http.+(\s)?)|((@\w*\d*(\s)?))' ##Removes url and user handles -- @xyz
corp = []
for tweet in tweets:
    tweet_clean = re.sub(pattern, '', tweet, flags=re.MULTILINE)
    corp.append(tweet_clean)

In [230]:
positive_sum = []
negative_sum = []
averagesenti = []
pos_neg_diff = []

for line in corp:
    #print(line)
    token = tokenizer_regex.tokenize(line)
    #print("token_regex: ",token)
    token = [word for word in token if word not in stopwords.words('english')]
    token = [lmtzr.lemmatize(word) for word in token]
    #print("token: ",token)
    pos_scores=[]
    neg_scores=[]
    pos = 0
    neg = 0
    for word in token:
        #print(word)
        a,b = 0, 0
        syn = wordnet.synsets(word)
        for sy in syn:
            senti = swn.senti_synset(sy.name())
            a+=senti.pos_score()
            b+=senti.neg_score()
        if len(syn) > 0:
            a = a/len(syn)
            b = b/len(syn)
        pos+=a
        neg+=b
        pos_scores.append(pos)
        neg_scores.append(neg)
    if(len(pos_scores)!=0):
        max_pos =max(pos_scores)
        max_neg = max(neg_scores)
        pos_sum = sum(pos_scores)
        neg_sum = sum(neg_scores)
        senti_avg = (pos_sum-neg_sum)/len(token)
    else:
        max_pos = 0
        max_neg =0
        pos_sum = 0
        neg_sum =0
        senti_avg =0

    imbal = pos_sum - neg_sum
    
    positive_sum.append(pos_sum)
    negative_sum.append(neg_sum)
    averagesenti.append(senti_avg)
    pos_neg_diff.append(imbal)

In [ ]:
print("positive_sum: ", positive_sum)
print("negative_sum: ", negative_sum)
print("averagesenti: ", averagesenti)
print("pos_neg_diff: ", pos_neg_diff)

In [ ]:
def print_to_file(data, filename):
    fo = open(filename+".txt","w+")
    for line in data:
        fo.write(line)
        fo.write("\n")
    fo.close()